In [8]:
pip show langchain

Name: langchain
Version: 0.2.8
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\Users\Adhiraj\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

file_path = r'C:\Users\Adhiraj\Downloads\Corpus.pdf'

loader = PyPDFLoader(file_path)
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=80,
)
doc = text_splitter.split_documents(pages)


In [25]:
len(doc)

47

In [5]:
print(doc[2])

page_content='and
when
guests
travel
home
to
share
great
wine
and
food
with
friends
and
family.
For
the
owners
and
staff,
it’s
all
about
creating
a
community
to
share
their
love
of
wine,
the
arts,
and
everything
that
makes
the
Napa
Valley
so
special.
CURRENT
GALLERY
FEATURE
JERMAINE
DANTÉ
Meet
our
new
resident
artist
at
the
Jessup
Gallery,
Jermaine
Danté.
As
a
resident
of
Napa
Valley,
Jermaine’s
artwork
is
characterized
by
its
bold
strokes
and
evocative
imagery,
serving
as
a
perfect
embodiment
of
the
symbiotic
relationship
between
art
and
wine.
A
certified
sommelier,
Jermaine
captures
the
essence
of
Wine
Country
in
many
of
his
pieces,
infusing
them
with
his
unique
perspective
and
artistic
flair.
Come
by
to
see
his
art
on
display
this
summer
and
enjoy
a
glass
of
wine
as
you
stroll
through
the
gallery.
Jessup
Cellars
and
Tasting
Gallery
is
located
at
6740
Washington
St.
at
the
North
end
of
Yountville
across
from
RH
Yountville
and
North
Block
Restaurant.
We
are
open
from
10AM
to
5:30PM
7


In [4]:
doc

[Document(metadata={'source': 'C:\\Users\\Adhiraj\\Downloads\\Corpus.pdf', 'page': 0}, page_content='Jessup\nCellars\nCorpus\nJessup\nCellars\nInformation\nJessup\nCellars\nhas\na\ncasual\nand\ninviting\natmosphere'),
 Document(metadata={'source': 'C:\\Users\\Adhiraj\\Downloads\\Corpus.pdf', 'page': 0}, page_content='inviting\natmosphere\nand\nwas\nthe\nfirst\ntasting\nroom\nopened\nin\nYountville\nin\n2003.\nYou\nhave\nthe'),
 Document(metadata={'source': 'C:\\Users\\Adhiraj\\Downloads\\Corpus.pdf', 'page': 0}, page_content='2003.\nYou\nhave\nthe\noption\nof\nsitting\ninside\nour\nstunning\nart\ngallery\nor\nyou\nmay\nchoose\nto\nenjoy\nthe'),
 Document(metadata={'source': 'C:\\Users\\Adhiraj\\Downloads\\Corpus.pdf', 'page': 0}, page_content='choose\nto\nenjoy\nthe\npatio\nwith\ngiant\numbrellas.\nWe\nalso\nhave\nspace\navailable\nfor\nprivate\ngroups\nand'),
 Document(metadata={'source': 'C:\\Users\\Adhiraj\\Downloads\\Corpus.pdf', 'page': 0}, page_content='private\ngroups\nand\nspec

In [26]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.runnables import RunnablePassthrough

llm = ChatOllama(model="phi3")
QueryPrompt = PromptTemplate(
    input_variables=["questions"],
    template = """
    You are AI language model assitant. Your task is to generate 3 different versions of the given user question to retrive relevant documents from a vector database.Try to include the year, prices and names where ever required. By generating multiple perspectives on user questions, your aim is to overcome the limitations of the distance based similarity search. Provide alternative questions separated by newlines. 
    Original Question: {question},
    """
)

In [27]:
from langchain_community.embeddings import OllamaEmbeddings

try:
    embedding_model = OllamaEmbeddings(model="nomic-embed-text")
    print("Embedding model loaded successfully.")
except Exception as e:
    print(f"Error loading embedding model: {e}")


Embedding model loaded successfully.


In [2]:
from langchain_chroma import Chroma

try:
    # Create an empty Chroma instance for testing
    db = Chroma()
    print("Chroma instance created successfully.")
except Exception as e:
    print(f"Error creating Chroma instance: {e}")


Chroma instance created successfully.


In [28]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

try:
    embedding_model = OllamaEmbeddings(model="nomic-embed-text")
    db = Chroma.from_documents(documents=doc, embedding=embedding_model, persist_directory = "db")
    print("Database created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Database created successfully.


In [23]:
db.delete_collection()

In [29]:
retriever = MultiQueryRetriever.from_llm(
    db.as_retriever(),
    llm,
    prompt = QueryPrompt
)
prompt_template = """ You help customers by answering their questions based on the context fetched by retriever. Make sure not to make any changes to the context. Answer the questions based ONLY the context and if you don't have an answer from the provided information than ask the user to contact the business dealer. The answer should not exceed 100 tokens. 
Given these texts:
-----
{context}
-----
Please answer the following question:
{question}

"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
)

In [ ]:
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_history = []
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [52]:
from langchain_community.document_transformers import LongContextReorder
question = "What is different about your 2018 Rougette versus the 2019?"
docs = retriever.invoke(question)
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)
chain = create_stuff_documents_chain(llm, prompt)
response = chain.invoke({"context": reordered_docs, "question": question})

In [53]:
print(response)

 As an AI developed before September 2016 and with no specific data on a wine named "Graziella" or one from Jessup Cellars released in that timeframe, I cannot provide accurate details about these wines. However, if this were real information provided for comparing two vintages of a fictional or unreleased wine like the 2018 Rougette and the 2table:

-----
When analyzing differences between the 2018 Rougette and the more recent 2019 iteration, we can expect to observe nuances in flavor profile development due to variations in weather conditions that each vintage experienced. The terroir of a particular year profoundly influences its resultant wine's character; thus, it is reasonable to assume these wines would exhibit distinct qualities reflective of their unique growing seasons and climatic circumstances during harvest time.

In the 2018 Rougette, we might have encountered a wine that leaned towards richer fruit flavors with possibly more pronounced tannins due to longer sun exposure 

In [47]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [38]:
chain.invoke("What is your best red wine?")

" As an AI developed by Microsoft, I don't have personal experiences or taste preferences. However, among many popular choices known for their quality and versatty nature are wines like Cabernet Sauvignon from Napa Valley, Malbec from Mendoza region in Argentina, Merlot also often hailing from places such as Bordeaux, France or Sonoma County in California; Shiraz/Syrah which comes from regions including Barossa Valley and McLaren Vale in Australia. The best red wine for you would significantly depend on your personal taste preference like body, tannin levels, acidity, sweetness etc., as well the food pairings or occasion it's going to be paired with."

In [31]:
chain.invoke("What makes Jessup Cellars unique compared to other tasting rooms in Yountville?") 

' The uniqueness of Jessup Cellars is a combination of their exclusive membership options, personalized wine selections curated by experts on behalf of each member and special events. Members are provided with an invitation-only experience where they receive three bottles of selected wines four times per year which can be shipped to them or picked up from the cellar premises depending upon their preference, offering flexibility that other tasting rooms may not provide in Yountville. The curated wine selection and personal touch adds a unique layer making Jessup Cellars stand out among its competitors by promoting an intimate connection between customer preferences, individualized selections of wines from the most requested list as well as new releases with members first knowledge about them while providing discounts on all their current release orders. Additionally, they offer special events for Members only which creates a sense of exclusivity and camaraderie among club members that m

In [19]:
chain.invoke("Can you tell me more about your 2022 Chardonnay?") 

' I\'m sorry, but based on the information provided in that document snippet from "Juel" Blend at Jessup Cellars, it does not appear to be a wine by them. The blend mentioned is composed of varietals primarily grown and sourced from Napa Valley regions like Truchard Vineyard, indicating different wineries with their own unique products in the market that might offer such specific Chardonnay variations for 2022 or earlier vintages.\n\nAt Jessup Cellars specifically mentioned on the document snippet are a Cabernet Sauvignon and Merlot blend along with Petite Syrah, Picpoul de Pétrolius (Sauvignon blanc), Malbec featuring an older Napa Valley bottling from 2019. There is no mention of Chardonnay in the provided document snippet either as part of their current wine offerings or a specific release for 2in your question, such as "Juel" Blend.\n\nFor accurate details regarding Jessup Cellars\' blends and vintages including any potential entry-level whites like Chardonnay that may have been re

In [20]:
chain.invoke("What varietal grapes are in your Rougette wine?") 

" The document provided does not specify any details regarding a specific variety of grape used for creating the Rougette wine. Therefore, it is impossible to extract this information from the given text. It would be necessary to have further detailed descriptions or product labels indicating exact grape varieties included in your Rougette wine selection at Jessup Cellars 6 and so on if you are seeking that kind of specificity about their wines' composition."

In [ ]:
# Quantize the embeddings to INT8
from sentence_transformers.quantization import quantize_embeddings

# Quantize embeddings
quantized_embeddings = quantize_embeddings(
    db,
    precision="int8",
    calibration_embeddings=embedding_model
)

In [ ]:
from langchain_core.runnables import chain

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):
    """Search over a database of job records."""

    queries: list[str] = Field(
        ...,
        description="Distinct queries to search for",
    )

from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

output_parser = PydanticToolsParser(tools=[Search])

structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm
@chain
async def custom_chain(question):
    response = await query_analyzer.ainvoke(question)
    docs = []
    for query in response.queries:
        new_docs = await retriever.ainvoke(query)
        docs.extend(new_docs)
    # You probably want to think about reranking or deduplicating documents here
    # But that is a separate topic
    return docs